# AJUSTE DE HIPER-PARÁMETROS EN XGBOOST - PARTE 1: CLASIFICACIÓN

Aunque XGBoost es un conjunto de modelos robustos, diseñados para reducir el "overfitting" en comparación con los modelos Gradient Boosting, siempre es recomendable ajustar los hiperparámetros del modelo para intentar reducir aún más su overfitting y a la vez aprovechar al máximo su desempeño.

El éxito de este ajuste de hiper-parámetros siempre dependerá tanto de las características del set de datos que estemos usando como de la misma forma como realicemos el ajuste y esto implica que no necesariamente siempre lograremos mejorar el desempeño del modelo.

## 1. El problema a resolver

En esta primera parte de la lección realizaremos la afinación de hiper-parámetros del clasificador XGBoost implementado en la lección 6.

## 2. Estrategia de ajuste sugerida

Dado el elevado número de hiper-parámetros que se pueden afinar se sugiere únicamente afinar aquellos que usualmente tienen mayor impacto en el desempeño ($\eta$, $\gamma$, $\lambda$, máxima profundidad de los árboles -'max_depth'- y
porcentaje de columnas a usar durante el entrenamiento -'colsample_bytree'-).

Teniendo esto en cuenta, la estrategia sugerida es la misma que usamos por ejemplo para afinar los hiper-parámetros de los árboles de decisión o de los bosques aleatorios, aunque con algunas ligeras diferencias:

![](grid-search-clasificacion.png)

Aunque podemos usar herramientas como GridSearchCV (vista en detalle en el curso de Árboles de Decisión) en este ejemplo veremos la implementación manual del algoritmo anterior para afinar el modelo de clasificación implementado en la lección 6.

## 3. Ajuste de hiper-parámetros para clasificación

Comencemos leyendo y pre-procesando el mismo set de datos usado en la lección 6. En este caso usaremos únicamente el set de entrenamiento para afinar los hiper-parámetros y el set de prueba para, como su nombre lo indica, poner a prueba el modelo afinado con datos totalmente nuevos:

In [1]:
# Leer dataset
import pandas as pd
from sklearn.model_selection import train_test_split

RUTA = '/Users/miguel/Library/CloudStorage/GoogleDrive-miguel@codificandobits.com/My Drive/02-CODIFICANDOBITS.COM/04-Academia/01-Cursos/30-2024-12-XGBoost/data/'
df = pd.read_csv(RUTA + 'dataset_prestamos_clasif.csv')

# Representar las variables categóricas como tipo "category"
# (género, educación, propietario, prestamo_uso, impagos previos)
cols_cat = df.select_dtypes(include='object').columns
for col in cols_cat:
    df[col] = df[col].astype("category")

df.info()

# Crear sets X y Y y partir dataset
X = df.iloc[:,:-1] # Características (variables predictoras)
Y = df.iloc[:,-1] # Variable a predecir

# Crear sets de entrenamiento y prueba (la validación se tomará del set de entrenamiento)
x_tr, x_ts, y_tr, y_ts = train_test_split(X, Y, train_size=0.8, random_state=123)

print('Tamaño set de entrenamiento: ' , x_tr.shape, y_tr.shape)
print('Tamaño set de prueba: ', x_ts.shape, y_ts.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19638 entries, 0 to 19637
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   edad                        19638 non-null  float64 
 1   genero                      19638 non-null  category
 2   educacion                   19638 non-null  category
 3   ingresos                    19638 non-null  float64 
 4   annos_exp_laboral           19638 non-null  int64   
 5   propietario                 19638 non-null  category
 6   prestamo_monto_solicitado   19638 non-null  float64 
 7   prestamo_uso                19638 non-null  category
 8   prestamo_tasa               19638 non-null  float64 
 9   prestamo_pctj_ing           19638 non-null  float64 
 10  annos_historial_crediticio  19638 non-null  float64 
 11  ptj_crediticio              19638 non-null  int64   
 12  impagos_previos             19638 non-null  category
 13  estado_prestamo 

A continuación implementaremos la función para realizar la validación cruzada. En este caso la idea es usar como métrica personalizada una proveniente de Scikit-Learn:

In [2]:
# Validación cruzada con métrica personalizada y early stopping

import xgboost as xgb
from sklearn.metrics import accuracy_score

def cv_clf_personalizada(dtrain, custom_metrics, params):
    '''Validación cruzada para clasificación con early stopping
       y métrica de desempeño personalizada
    
    Entradas:
    - dtrain: datos de entrenamiento en formato DMatrix
    - custom_metrics: métrica de desempeño personalizada
    - params: hiper-parámetros del modelo XGBoost a validar

    Salida:
    - cv_res: DataFrame de Pandas con los resultados de la validación
    '''

    cv_res = xgb.cv(
        params=params,
        dtrain=dtrain,
        num_boost_round=1000,           # Número inicial de árboles a entrenar
        early_stopping_rounds=10,       # Criterio de parada temprana con base en la métrica personalizada
        nfold=5,                        # Número de particiones
        seed=123,                       # Semilla generador aleatorio
        custom_metric=custom_metrics,   # *** MÉTRICA DE EVALUACIÓN PERSONALIZADA ***
        maximize=True,                  # Maximizar la exactitud
        as_pandas=True                  # Retornar resultados como DataFrame de Pandas
    )

    return cv_res

Y usemos la función anterior para crear un modelo base y estimar su desempeño. Este modelo base nos servirá como referencia para determinar si el modelo afinado tiene o no un mejor desempeño:

In [3]:
# Calcular e imprimir en pantalla desempeño modelo base

# Modelo base
parametros = {
    'booster': 'gbtree',           # Usar árboles tipo gradient boosting
    'eta': 0.1,                    # Tasa de aprendizaje
    'gamma': 0,
    'lambda': 0,
    'max_depth': 6,                # Profundidad máxima de los árboles
    'colsample_bytree': 0.8,       # Porcentaje de columnas a muestrear
    'objective': 'binary:logistic',# Pérdida
    'random_state': 42             # Semilla generador aleatorio
    }

# Métrica personalizada
def custom_metrics_clf(preds, dtrain):
    labels = dtrain.get_label()  # Valores a predecir
    preds_binary = [1 if p > 0.5 else 0 for p in preds]  # De probabilidades a predicciones
    acc = accuracy_score(labels, preds_binary)
    return 'exactitud', acc  # Retornar el nombre y el valor de la métrica

# CV personalizada
dtrain = xgb.DMatrix(data=x_tr, label=y_tr, enable_categorical=True)
cv_res = cv_clf_personalizada(dtrain, custom_metrics=custom_metrics_clf, params = parametros)

# Imprimir en pantalla
best_round = cv_res['test-exactitud-mean'].idxmax()
clf_base_train = cv_res['train-exactitud-mean'].iloc[best_round]
clf_base_test = cv_res['test-exactitud-mean'].iloc[best_round]

print('DESEMPEÑO MODELO BASE - VALIDACIÓN CRUZADA: ')
print('='*50)
print(f'   Exactitud train: {100*clf_base_train:.1f}%')
print(f'   Exactitud test: {100*clf_base_test:.1f}%')

DESEMPEÑO MODELO BASE - VALIDACIÓN CRUZADA: 
   Exactitud train: 92.8%
   Exactitud test: 90.4%


Ahora implementaremos la función para realizar la afinación de hiper-parámetros, haciendo uso de la validación cruzada implementada anteriormente:

In [4]:
# Función para afinar hiper-parámetros
from sklearn.model_selection import ParameterGrid # Para obtener todas las combinaciones de hiper-parámetros

def afinar_hiperparams_clf(dtrain, grilla, parametros, custom_metrics):
    '''Ajuste de hiper-parámetros usando búsqueda exhaustiva (Grid Search)
    
    Entradas:
    - dtrain: datos de entrenamiento en formato DMatrix
    - grilla: grilla de hiper-parámetros definida por el usuario
    - parametros: hiper-parámetros del modelo XGBoost que no serán ajustados
    - custom_metrics: métrica de desempeño personalizada

    Salida:
    - mejor_puntaje: mejor métrica de desempeño obtenida tras la afinación
    - mejores_hiperparams: mejor conjunto de hiper-parámetros obtenido tras la afinación  
    '''
    
    # 1. Inicializar mejores hiper-parámetros y mejor puntaje (exactitud)
    mejores_hparams = None
    mejor_puntaje = 0

    # 2. Realizar ajuste de hiper-parámetros
    N = len(ParameterGrid(grilla))

    for i, combinacion in enumerate(ParameterGrid(grilla)):
        print(f'Combinación {i+1}/{N}...')

        # Definir hiper-parámetros
        params = combinacion | parametros

        # Validación cruzada con early-stopping y métrica "accuracy"
        cv_res = cv_clf_personalizada(dtrain, custom_metrics, params=params)

        # Extraer mejor desempeño para los "folds" de prueba
        puntaje_set_prueba = cv_res['test-exactitud-mean'].max()

        if puntaje_set_prueba > mejor_puntaje:
            mejor_puntaje = puntaje_set_prueba
            mejores_hparams = combinacion
            print(f'    Mejor puntaje hasta el momento (folds de prueba): {mejor_puntaje:.5f}')
        
    # Retornar mejor puntaje y mejores hiper-parámetros
    return mejor_puntaje, mejores_hparams


Y en este punto ya estamos listos para realizar la afinación de hiper-parámetros. Simplemente creamos la grilla de hiper-parámetros y definimos igualmente los hiper-parámetros que NO se afinarán, e introducimos ambos como argumentos de la función anterior:

In [5]:
# Realizar afinación con la función anterior
# Crear grilla de hiper-parámetros
grilla_hparams = {
    'eta': [0.4, 0.5, 0.6],     # Tasas de aprendizaje
    'gamma': [0, 0.1, 0.2],     # Hiper-parámetro 𝜸
    'lambda': [1.5, 2, 2.5],    # Hiper-parámetro λ
    'max_depth': [2, 3, 4],     # Máxima profundidad del árbol
    'colsample_bytree': [0.7, 0.8, 0.9], # Porcentaje de muestreo de las columnas
}

# Parámetros que no se ajustarán
params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'random_state': 42,   
    'n_jobs': -1
}

best_score, best_params = afinar_hiperparams_clf(dtrain, grilla_hparams, params, custom_metrics_clf)

# Imprimir mejores hiper-parámetros y mejor puntaje
print("Mejores hiper-parámetros:", best_params)
print("Mejor exactitud:", f'{100*best_score:.1f}%')


Combinación 1/243...
    Mejor puntaje hasta el momento (folds de prueba): 0.90560
Combinación 2/243...
    Mejor puntaje hasta el momento (folds de prueba): 0.90757
Combinación 3/243...
Combinación 4/243...
Combinación 5/243...
Combinación 6/243...
Combinación 7/243...
Combinación 8/243...
Combinación 9/243...
    Mejor puntaje hasta el momento (folds de prueba): 0.90764
Combinación 10/243...
Combinación 11/243...
    Mejor puntaje hasta el momento (folds de prueba): 0.90827
Combinación 12/243...
    Mejor puntaje hasta el momento (folds de prueba): 0.90898
Combinación 13/243...
Combinación 14/243...
Combinación 15/243...
Combinación 16/243...
Combinación 17/243...
Combinación 18/243...
Combinación 19/243...
Combinación 20/243...
Combinación 21/243...
Combinación 22/243...
Combinación 23/243...
Combinación 24/243...
    Mejor puntaje hasta el momento (folds de prueba): 0.90987
Combinación 25/243...
Combinación 26/243...
Combinación 27/243...
Combinación 28/243...
Combinación 29/243...

Y ahora estimaremos el desempeño del modelo afinado tanto para entrenamiento como para prueba usando la función para realizar la validación cruzada. En este caso usaremos como parámetros tanto los parámetros que no se afinan como los mejores hiper-parámetros que acabamos de encontrar:

In [6]:
# Mostrar desempeño del modelo afinado

# Modelo afinado
parametros = best_params | params

# CV personalizada
cv_res = cv_clf_personalizada(dtrain, custom_metrics_clf, params = parametros)

# Imprimir en pantalla
best_round = cv_res['test-exactitud-mean'].idxmax()
clf_afinado_train = cv_res['train-exactitud-mean'].iloc[best_round]
clf_afinado_test = cv_res['test-exactitud-mean'].iloc[best_round]

print('DESEMPEÑO MODELO AFINADO - VALIDACIÓN CRUZADA: ')
print('='*50)
print(f'   Exactitud train: {100*clf_afinado_train:.1f}%')
print(f'   Exactitud test: {100*clf_afinado_test:.1f}%')

DESEMPEÑO MODELO AFINADO - VALIDACIÓN CRUZADA: 
   Exactitud train: 94.6%
   Exactitud test: 91.0%


Y hemos logrado pasar de un modelo con desempeño entrenamiento/prueba 92.8%/90.4% a uno con desempeño de 94.6%/91.0%.

Aunque esta afinación ha incrementado el desempeño con los datos de prueba también ha incrementado ligeramente el "overfitting" y esto se podría mejorar probando con una grilla hiper-parámetros más amplia.

Sin embargo, también debemos tener en cuenta que el incremento en el desempeño (90.4% a 91.0%) ha sido marginal y esto muestra que XGBoost es realmente un modelo robusto y que, generalmente, las mejoras tras la afinación de hiper-parámetros serán marginales.

Por último, podemos entrenar el modelo afinado y generar predicciones con datos totalmente nuevos (el set de prueba, que no hemos usado hasta el momento):

In [7]:
# Entrenar modelo afinado
from xgboost import XGBClassifier

params = parametros | {'enable_categorical':True}
clf_afinado = XGBClassifier(**params)
clf_afinado.fit(x_tr,y_tr) # Se recomienda agregar early stopping

# Y generar predicciones sobre datos nuevos
preds = clf_afinado.predict(x_ts)
preds



array([0, 0, 0, ..., 0, 1, 0])